In [8]:
import pymysql
import csv
from datetime import datetime
import re

# create a connection object 'conn'
conn = pymysql.connect(host="localhost", # your host, localhost for your local machine
                     user="wodbuser", # your username, usually "root" for localhost
                      passwd="wodbpassword", # your password
                      db="wodb", connect_timeout=5) # name of the data base; world comes inbuilt with mysql

In [38]:
# field names
# ['Order ID', 'Order-status', 'Brand', 'Kitchen', 
# 'Order-relay-time(ordered time)', 'Order-acceptance-time <placed_time>', 
# 'Order-delivery-time', 'Total-bill-amount <bill>', 'Tax Restaurant', 'Item-SGST', 
# 'Item-CGST', 'Item-IGST', 'PackagingCharge-SGST', 'PackagingCharge-CGST', 'PackagingCharge-IGST', 
# 'ServiceCharge-SGST', 'ServiceCharge-CGST', 'ServiceCharge-IGST', 'Item-GST-Inclusive', 
# 'Packaging_GST_Inclusive', 'ServiceCharge-GST-inclusive', 'Restaurant Discount', 'Packing-charge', 
# 'Cancelled reason', 'Food-prepared <Yes/No>', 'Order-Cancellation-time', 'Edited-status', 'Item-count', 
# 'MOU type', 'Cancellation-responsible-entity', 'Restaurant-bear', 
# 'Item1-name_reward_type_quantity_price+Variants+Addons', '', '', '', '']
# Tables - 
# AggOrder
# orderId status('Order-status') billNumber('Order ID') slot kitchen('Kitchen') paymentMode source('Swiggy') brand('Brand') 
# orderComment orderStartTime ('Order-relay-time(ordered time)')
# orderEndTime('Order-delivery-time') subtotal total('Total-bill-amount <bill>') createdAt updatedAt taxExempted
# AggOrderItems
# orderItemId orderId itemName('name') itemPrice('price') itemTotalPrice('price') itemQuantity('quantity') itemType totalDiscount totalTax
# woProductId woProductSkuId createdAt updatedAt
# AggOrderCustomer
# orderCustomerId orderId address1 userId city countryCode country fullName mobile createdAt updatedAt

def getItemmap(itemstring):
    # e.g. - Red Velvet In A Jar_NA_1_12,Punchy Pineapple Cake in Jar_NA_1_12
    itemmap = {}
    if (itemstring == ''):
        return itemmap
    addons = itemstring.split('++')
    itemstring1 = ''
    if (len(addons) == 0):
        return itemmap
    itemstring1 = addons[0]
    variants = itemstring1.split('+')  
    itemstring2 = variants[0]     
    itemtokens = itemstring2.split('_')
    
    if (len(itemtokens) == 4):
        itemmap['name'] = itemtokens[0].strip()
        itemmap['reward_type'] = itemtokens[1].strip()
        itemmap['quantity'] = itemtokens[2].strip()
        itemmap['price'] = itemtokens[3].strip()
    return itemmap
def insertSwiggyOrderItem(cur, orderid, billno, itemstrings):
    # print (itemstrings)
    items = list(map(lambda x: getItemmap(x), itemstrings))
    for item in items:
        if (len(item) == 4):
            sql = "".join(["insert into AggOrderItems (orderId, billNumber, itemName, itemPrice, itemQuantity, createdAt, updatedAt) values('", orderid ,"','", billno, "','", item['name'], "','", item['price'], "','", item['quantity'],"', current_timestamp(), current_timestamp())"])
            cur.execute(sql)
def insertSwiggyOrder(header, row, conn):
    #print ('\n')
    #print (row)
    itemindex = header.index("Item1-name_reward_type_quantity_price+Variants+Addons")
    maxindex = len(header) - 1
    colmap = {'orderId':'S' + row[header.index('Order ID')], 'billNumber':row[header.index('Order ID')], 'status':row[header.index('Order-status')],'kitchen':row[header.index('Kitchen')], 'source':'Swiggy', 'brand':row[header.index('Brand')], 'orderStartTime':row[header.index('Order-relay-time(ordered time)')], 'orderEndTime':row[header.index('Order-delivery-time')], 'total':row[header.index('Total-bill-amount <bill>')]} 
    print (colmap)
    with conn.cursor() as cur:
        tempdatestart = colmap['orderStartTime']
        tempdateend = colmap['orderEndTime']
        datestart = datetime.strptime(tempdatestart,  '%d/%m/%y %H:%M')
        dateend = datetime.strptime(tempdateend,  '%d/%m/%y %H:%M')
        sql = "".join(["insert into AggOrder (orderId, billNumber, status, kitchen, source, brand, orderStartTime, orderEndTime, total, createdAt, updatedAt) values('", colmap['orderId'], "','" , colmap['billNumber'], "','", colmap['status'], "','",colmap['kitchen'],"','",colmap['source'],"','",colmap['brand'],"','", dateend,"','",dateend,"','",colmap['total'],"', current_timestamp(), current_timestamp())"])
        #print (sql)
        cur.execute(sql)
        insertSwiggyOrderItem(cur, colmap['orderId'],colmap['billNumber'], row[itemindex:maxindex])
    conn.commit()
def insertZomatoOrder(header, row, conn):
    print ('\n')
    # print (header)
    orderidindex = -1
    try:
        orderidindex = header.index('Order ID')
    except:
        try:
            orderidindex = header.index('\ufeffOrder Id')
        except:
            orderidindex = -1
    if (orderidindex == -1):
        return
    
    # print (orderidindex)
    colmap = {'orderId':'Z' + row[orderidindex], 'billNumber':row[orderidindex], 'status':row[header.index('Delivery Status')],'kitchen':row[header.index('Kitchen')], 'source':'Zomato', 'brand':row[header.index('Brand')], 'orderStartTime':row[header.index('Order Date')], 'orderEndTime':row[header.index('Order Date')], 'total':row[header.index('Amount')]} 
    with conn.cursor() as cur:
        date = datetime.strptime(colmap['orderStartTime'],  '%I:%M %p, %B %d %Y')
        date = date.strftime('%Y-%m-%d %H:%M:%S')
        sql = "".join(["insert into AggOrder (orderId, billNumber, status, kitchen, source, brand, orderStartTime, orderEndTime, total, createdAt, updatedAt) values('", colmap['orderId'], "','" , colmap['billNumber'], "','", colmap['status'], "','",colmap['kitchen'],"','",colmap['source'],"','",colmap['brand'],"','", date,"','",date,"','",colmap['total'],"', current_timestamp(), current_timestamp())"])
        print (sql)
        cur.execute(sql)
        #insertSwiggyOrderItem(cur, colmap['orderId'],colmap['billNumber'], row[itemindex:maxindex])
    conn.commit()
def insertFoodpandaOrderItem(cur, orderid, billno, itemstrings):
    # print (itemstrings)
    items = itemstrings.split(';')
    for item in items:
        if (len(item) > 1):
            itemqtytemp = re.findall('^\d', item)
            itemqty = itemqtytemp[0]
            itemnametemp = re.findall('\d\s(.*)', item)
            itemname = itemnametemp[0]
            sql = "".join(["insert into AggOrderItems (orderId, billNumber, itemName, itemPrice, itemQuantity, createdAt, updatedAt) values('", orderid ,"','", billno, "','", itemname, "',0,'", itemqty,"', current_timestamp(), current_timestamp())"])
            #print (sql)
            cur.execute(sql)
def insertFoodpandaOrder(header, row, conn):
    print ('\n')
    print (header)
    # ['\ufeffORDER CODE', 'RESTAURANT CODE', 'RESTAURANT NAME', 'FOOD COST', 'CONTAINER PRICE', 'RESTAURANT DELIVERY FEE', 'RESTAURANT SERVICE FEE', 'RIDER PAYS', 'PAYMENT TYPE', 'ITEMS', 'ORDER ITEMS', 'EXPECTED DELIVERY DATE', 'EXPECTED DELIVERY TIME', 'EXPEDITION TYPE', 'STATUS', 'CANCELLATION REASON']
    orderidindex = -1
    try:
        orderidindex = header.index('ORDER CODE')
    except:
        try:
            orderidindex = header.index('\ufeffORDER CODE')
        except:
            orderidindex = -1
    if (orderidindex == -1):
        return
    # WarmOven ( BTM 2nd Stage )
    restname = row[header.index('RESTAURANT NAME')]
    brand = re.findall('^[^\s\(]*', restname)
    kitchen = re.findall('\(\s(.+)\s\)', restname)
    #print (brand[0])
    #print (kitchen[0])
    tempdate = row[header.index('EXPECTED DELIVERY DATE')] + row[header.index('EXPECTED DELIVERY TIME')]
    date = datetime.strptime(tempdate,  '%d/%m/%y%H:%M %p')
    date = date.strftime('%Y-%m-%d %H:%M:%S')
    # 19/09/18
    # 5:15 PM
    # print (orderidindex)
    colmap = {'orderId':'F' + row[orderidindex], 'billNumber':row[orderidindex], 'status':row[header.index('STATUS')],'kitchen':kitchen[0], 'source':'Foodpanda', 'brand':brand[0], 'total':row[header.index('FOOD COST')], 'orderItems':row[header.index('ORDER ITEMS')]} 
    with conn.cursor() as cur:
        sql = "".join(["insert into AggOrder (orderId, billNumber, status, kitchen, source, brand, orderStartTime, orderEndTime, total, createdAt, updatedAt) values('", colmap['orderId'], "','" , colmap['billNumber'], "','", colmap['status'], "','",colmap['kitchen'],"','",colmap['source'],"','",colmap['brand'],"','", date,"','",date,"','",colmap['total'],"', current_timestamp(), current_timestamp())"])
        # print (sql)
        cur.execute(sql)
        # ORDER ITEMS
        insertFoodpandaOrderItem(cur, colmap['orderId'],colmap['billNumber'], colmap['orderItems'])
    conn.commit()    
def insertOrder(header, row, conn, orderalso):
    print ('\n')
    # print (header)
    # ['Date', 'Time', 'Bill Number', 'Reference Number', 'Outlet Name', 'Order Status', 'Order Type', 'Order Source', 'Subtotal', 'Item Name', 'Item Quantity', 'Item Total', 'Item Complementary', 'Item Complementary Quantity', 'Item Discount', 'Item Taxes', 'Item Charges'] 90
    # ['2018-11-14', '12:01 AM', '', '25209096113', 'Warm Oven - Whitefield', 'DELIVERED', 'Delivery', 'Swiggy', 129.0, 'Death By Chocolate In A Jar', 1.0, 129.0, 'No', 0.0, 0.0, 0.0, 0.0]
    orderidindex = -1
    try:
        orderidindex = header.index('Reference Number')
    except:
        orderidindex = -1
    if (orderidindex == -1):
        return
    colmap = {'orderId':row[orderidindex], 'billNumber':row[header.index('Bill Number')], 'status':row[header.index('Order Status')],'brandANDkitchen':row[header.index('Outlet Name')], 'source':row[header.index('Order Source')], 'total':row[header.index('Subtotal')], 'itemName':row[header.index('Item Name')], 'itemQty':row[header.index('Item Quantity')], 'itemTotal':row[header.index('Item Total')]} 
    with conn.cursor() as cur:
        date = row[header.index('Date')] + ' ' + row[header.index('Time')] 
        date = datetime.strptime(date, '%Y-%m-%d %I:%M %p')
        date = date.strftime('%Y-%m-%d %H:%M:%S')
        brandAndKitchen = colmap['brandANDkitchen']
        bAk = brandAndKitchen.split('-')
        brand = bAk[0]
        kitchen = bAk[1]
        if (orderalso):
            sql = "".join(["insert into AggOrder (orderId, billNumber, status, kitchen, source, brand, orderStartTime, orderEndTime, total, createdAt, updatedAt) values('", colmap['orderId'], "','" , colmap['billNumber'], "','", colmap['status'], "','",kitchen,"','",colmap['source'],"','",brand,"','", date,"','",date,"','",str(colmap['total']),"', current_timestamp(), current_timestamp())"])
            print (sql)
            cur.execute(sql)
        sql = ''.join(["insert into AggOrderItems (orderId, billNumber, itemName, itemPrice, itemQuantity, createdAt, updatedAt) values('", colmap['orderId'],"','", colmap['billNumber'], "','", colmap['itemName'], "','" + str(colmap['itemTotal']) + "','", str(colmap['itemQty']),"', current_timestamp(), current_timestamp())"])
        print (sql)
        cur.execute(sql)       

In [39]:
import xlrd
import csv

messages = []

book = xlrd.open_workbook('Order Wise Item Sales Dataffbe42bf-d61f-446b-9924-fa640761b953.xlsx', on_demand = True)
first_sheet = book.sheet_by_index(0)
header = first_sheet.row_values(19)
rows = first_sheet.nrows
columns = len(header)
print (header)
print (rows)
i = 20
prevrow = header
orderid = ""
orderalso = True
while i < rows:
    row = first_sheet.row_values(i)
    if (row[0] == ''):
        j = 0
        while (j < columns):
            if (row[j] == ''):
                row[j] = prevrow[j]
            j = j + 1
    else:
        prevrow = row
    i = i + 1
    # print (row)
    if (orderid != row[header.index('Reference Number')]):
        orderalso = True
    else:
        orderalso = False
    orderid = row[header.index('Reference Number')]
    insertOrder(header, row, conn, orderalso)
conn.commit() 

with open('Swiggy_14th.csv') as csvfile:
    reader = csv.reader(csvfile)
    i = 0
    header = []
    for row in reader:
        i+=1
        # print (row)
        if (i == 1):
            header = row
        #else:
            #insertSwiggyOrder(header, row, conn)
with open('Zomato_14th.csv') as csvfile:
    reader = csv.reader(csvfile)
    i = 0
    header = []
    for row in reader:
        i+=1
        if (i == 1):
            header = row
        #else:
            #insertZomatoOrder(header, row, conn)
with open('Foodpanda_14th.csv') as csvfile:
    reader = csv.reader(csvfile)
    i = 0
    header = []
    for row in reader:
        i+=1
        if (i == 1):
            header = row
        #else:
            #insertFoodpandaOrder(header, row, conn)

['Date', 'Time', 'Bill Number', 'Reference Number', 'Outlet Name', 'Order Status', 'Order Type', 'Order Source', 'Subtotal', 'Item Name', 'Item Quantity', 'Item Total', 'Item Complementary', 'Item Complementary Quantity', 'Item Discount', 'Item Taxes', 'Item Charges']
90


insert into AggOrder (orderId, billNumber, status, kitchen, source, brand, orderStartTime, orderEndTime, total, createdAt, updatedAt) values('25209096113','','DELIVERED',' Whitefield','Swiggy','Warm Oven ','2018-11-14 00:01:00','2018-11-14 00:01:00','129.0', current_timestamp(), current_timestamp())


IntegrityError: (1062, "Duplicate entry '25209096113' for key 'PRIMARY'")

In [9]:
item_count = 0
with conn.cursor() as cur:
    cur.execute('insert into testpy (id, name) values("5", "Joe")')
    conn.commit()
    cur.execute("select * from testpy")
    for row in cur:
        item_count += 1
        print (row)
conn.commit()

('1', 'Joe')
('2', 'Joe')
('3', 'Joe')
('4', 'Joe')
('5', 'Joe')


In [4]:
            # 19/09/18
    # 5:15 PM
    # datetime.strptime("21/11/06 16:30", "%d/%m/%y %H:%M")
        #date = datetime.strptime('September 14 2018',  '%B %d %Y')
        # ' does not match format '%d/%m/%y %H:%M'
        # 2018-11-14', '12:01 AM # %p
date = datetime.strptime('2018-11-14 12:01 AM',  '%Y-%m-%d %I:%M %p')
print (date)

2018-11-14 00:01:00


In [2]:

import json
import urllib.parse
import email.parser as Parser

In [11]:
import email
from email import policy
from email.parser import BytesParser
with open('16.xlsx', 'rb') as fp:  # select a specific email file from the list
    emailString = BytesParser(policy=policy.default).parse(fp)
text = emailString.get_body(preferencelist=('plain')).get_content()
if emailString.is_multipart():
    for part in emailString.walk():
        if part.get_content_type() == 'text/plain':
            body = part.get_payload()
            print (body)
        else:
            body = emailString.get_payload()
            print (body)


print(text)  # print the email content

[<email.message.EmailMessage object at 0x10587d5f8>, <email.message.EmailMessage object at 0x10587d748>]
[<email.message.EmailMessage object at 0x10587d5f8>, <email.message.EmailMessage object at 0x10587d748>]
---------- Forwarded message ---------
From: Sumit Gupta <montygupta@gmail.com>
Date: Thu, Nov 15, 2018 at 2:58 PM
Subject: test
To: <limetray@cupcakebliss.in>


test

[<email.message.EmailMessage object at 0x10587d5f8>, <email.message.EmailMessage object at 0x10587d748>]
[<email.message.EmailMessage object at 0x10587d5f8>, <email.message.EmailMessage object at 0x10587d748>]
---------- Forwarded message ---------
From: Sumit Gupta <montygupta@gmail.com>
Date: Thu, Nov 15, 2018 at 2:58 PM
Subject: test
To: <limetray@cupcakebliss.in>


test



In [1]:
import email
from base64 import b64decode
import sys
from email.parser import Parser as EmailParser
from email.utils import parseaddr
from io import BytesIO

class NotSupportedMailFormat(Exception):
    pass

def parse_attachment(message_part):
    content_disposition = message_part.get("Content-Disposition", None)
    if content_disposition:
        dispositions = content_disposition.strip().split(";")
        if bool(content_disposition and dispositions[0].lower() == "attachment"):

            file_data = message_part.get_payload(decode=True)
            attachment = BytesIO(file_data)
            attachment.content_type = message_part.get_content_type()
            attachment.size = len(file_data)
            attachment.name = None
            attachment.create_date = None
            attachment.mod_date = None
            attachment.read_date = None

            for param in dispositions[1:]:
                name,value = param.split("=")
                name = name.lower()

                if name == "filename":
                    attachment.name = value
                elif name == "create-date":
                    attachment.create_date = value  #TODO: datetime
                elif name == "modification-date":
                    attachment.mod_date = value #TODO: datetime
                elif name == "read-date":
                    attachment.read_date = value #TODO: datetime
            return attachment

    return None

def parse(content):
    p = EmailParser()
    msgobj = p.parse(content)
    if msgobj['Subject'] is not None:
        decodefrag = decode_header(msgobj['Subject'])
        subj_fragments = []
        for s , enc in decodefrag:
            if enc:
                s = unicode(s , enc).encode('utf8','replace')
            subj_fragments.append(s)
        subject = ''.join(subj_fragments)
    else:
        subject = None

    attachments = []
    body = None
    html = None
    for part in msgobj.walk():
        attachment = parse_attachment(part)
        if attachment:
            attachments.append(attachment)
        elif part.get_content_type() == "text/plain":
            if body is None:
                body = ""
            body += unicode(
                part.get_payload(decode=True),
                part.get_content_charset(),
                'replace'
            ).encode('utf8','replace')
        elif part.get_content_type() == "text/html":
            if html is None:
                html = ""
            html += unicode(
                part.get_payload(decode=True),
                part.get_content_charset(),
                'replace'
            ).encode('utf8','replace')
    return {
        'subject' : subject,
        'body' : body,
        'html' : html,
        'from' : parseaddr(msgobj.get('From'))[1], # 名前は除いてメールアドレスのみ抽出
        'to' : parseaddr(msgobj.get('To'))[1], # 名前は除いてメールアドレスのみ抽出
        'attachments': attachments,
    }

In [9]:
import email
from email import policy
from email.parser import BytesParser
import xlrd

attachments = []
with open('8ek1sjml84t0dnfh0torns6amla0vah86j65so01', 'rb') as fp:  # select a specific email file from the list
    emailString = BytesParser(policy=policy.default).parse(fp)
text = emailString.get_body(preferencelist=('plain')).get_content()
if emailString.is_multipart():
    for part in emailString.walk():
        attachment = parse_attachment(part)
        if attachment:
            #wrapper = TextIOWrapper(attachment)
            # print (wrapper)
            # file_contents=xlsx_file.read().decode(encoding=file_enc, errors='replace')
            #book = xlrd.open_workbook(attachment.message_from_bytes(), on_demand = True)
            temporarylocation="testout.xlsx"
            with open(temporarylocation,'wb') as out: ## Open temporary file as bytes
                out.write(attachment.read())                ## Read bytes into file
            
           # book = pd.ExcelFile(attachment, engine='xlrd')
            book = xlrd.open_workbook("testout.xlsx", on_demand = True)
            first_sheet = book.sheet_by_index(0)
            procesSheet(book, conn)
    print (attachments)
    

['Date', 'Time', 'Bill Number', 'Reference Number', 'Outlet Name', 'Order Status', 'Order Type', 'Order Source', 'Subtotal', 'Item Name', 'Item Quantity', 'Item Total', 'Item Complementary', 'Item Complementary Quantity', 'Item Discount', 'Item Taxes', 'Item Charges']
656


insert into AggOrder (orderId, billNumber, status, kitchen, source, brand, orderStartTime, orderEndTime, total, createdAt, updatedAt) values('1628330660','','DELIVERED',' Sarjapur','Zomato','Indiana Burgers ','2018-11-16 00:08:00','2018-11-16 00:08:00','109.0', current_timestamp(), current_timestamp())
insert into AggOrderItems (orderId, billNumber, itemName, itemPrice, itemQuantity, createdAt, updatedAt) values('1628330660','','Veg Whopper','109.0','1.0', current_timestamp(), current_timestamp())


insert into AggOrder (orderId, billNumber, status, kitchen, source, brand, orderStartTime, orderEndTime, total, createdAt, updatedAt) values('1009172531','','DELIVERED',' Whitefield','Zomato','Indiana Burgers ','2018-11-



insert into AggOrderItems (orderId, billNumber, itemName, itemPrice, itemQuantity, createdAt, updatedAt) values('25436900804','','Box of 6','210.0','1.0', current_timestamp(), current_timestamp())


insert into AggOrder (orderId, billNumber, status, kitchen, source, brand, orderStartTime, orderEndTime, total, createdAt, updatedAt) values('25436919093','','DELIVERED',' Kaggdaspura','Swiggy','Warm Oven ','2018-11-16 15:18:00','2018-11-16 15:18:00','499.0', current_timestamp(), current_timestamp())
insert into AggOrderItems (orderId, billNumber, itemName, itemPrice, itemQuantity, createdAt, updatedAt) values('25436919093','','Eggless Chocolate Truffle Cake (500 gms)','499.0','1.0', current_timestamp(), current_timestamp())


insert into AggOrder (orderId, billNumber, status, kitchen, source, brand, orderStartTime, orderEndTime, total, createdAt, updatedAt) values('25437047994','','DELIVERED',' Kammanahalli','Swiggy','Indiana Burgers ','2018-11-16 15:20:00','2018-11-16 15:20:00','260.0',



insert into AggOrder (orderId, billNumber, status, kitchen, source, brand, orderStartTime, orderEndTime, total, createdAt, updatedAt) values('25453294672','','DELIVERED',' Domlur','Swiggy','Indiana Burgers ','2018-11-16 19:51:00','2018-11-16 19:51:00','561.0', current_timestamp(), current_timestamp())
insert into AggOrderItems (orderId, billNumber, itemName, itemPrice, itemQuantity, createdAt, updatedAt) values('25453294672','','Fries Combo (French Fries + Coke 250 ml)','208.0','2.0', current_timestamp(), current_timestamp())


insert into AggOrderItems (orderId, billNumber, itemName, itemPrice, itemQuantity, createdAt, updatedAt) values('25453294672','','Chicken Whopper Burger','146.0','1.0', current_timestamp(), current_timestamp())


insert into AggOrderItems (orderId, billNumber, itemName, itemPrice, itemQuantity, createdAt, updatedAt) values('25453294672','','Lamb Whopper Burger','167.0','1.0', current_timestamp(), current_timestamp())


insert into AggOrderItems (orderId, billN

insert into AggOrderItems (orderId, billNumber, itemName, itemPrice, itemQuantity, createdAt, updatedAt) values('25463089043','','Vanilla Cupcake (Pack of 6)','209.0','1.0', current_timestamp(), current_timestamp())


insert into AggOrderItems (orderId, billNumber, itemName, itemPrice, itemQuantity, createdAt, updatedAt) values('25463089043','','Ferrero Rocher Cupcake (Pack of 6)','289.0','1.0', current_timestamp(), current_timestamp())


insert into AggOrder (orderId, billNumber, status, kitchen, source, brand, orderStartTime, orderEndTime, total, createdAt, updatedAt) values('1498305852','','DELIVERED',' Kammanahalli','Zomato','Indiana Burgers ','2018-11-16 22:35:00','2018-11-16 22:35:00','69.0', current_timestamp(), current_timestamp())
insert into AggOrderItems (orderId, billNumber, itemName, itemPrice, itemQuantity, createdAt, updatedAt) values('1498305852','','Chocolate Milkshake','69.0','1.0', current_timestamp(), current_timestamp())


insert into AggOrder (orderId, billNumber,

In [5]:
import os
import sys
import uuid
import email
from email import policy
from email.parser import BytesParser
import xlrd
from base64 import b64decode
from email.parser import Parser as EmailParser
from email.utils import parseaddr
from io import BytesIO
import logging
import pymysql
from datetime import datetime
import re
import json
import urllib.parse  

def insertOrder(header, row, conn, orderalso):
    print ('\n')
    # print (header)
    # ['Date', 'Time', 'Bill Number', 'Reference Number', 'Outlet Name', 'Order Status', 'Order Type', 'Order Source', 'Subtotal', 'Item Name', 'Item Quantity', 'Item Total', 'Item Complementary', 'Item Complementary Quantity', 'Item Discount', 'Item Taxes', 'Item Charges'] 90
    # ['2018-11-14', '12:01 AM', '', '25209096113', 'Warm Oven - Whitefield', 'DELIVERED', 'Delivery', 'Swiggy', 129.0, 'Death By Chocolate In A Jar', 1.0, 129.0, 'No', 0.0, 0.0, 0.0, 0.0]
    orderidindex = -1
    try:
        orderidindex = header.index('Reference Number')
    except:
        orderidindex = -1
    if (orderidindex == -1):
        return
    colmap = {'orderId':row[orderidindex], 'billNumber':row[header.index('Bill Number')], 'status':row[header.index('Order Status')],'brandANDkitchen':row[header.index('Outlet Name')], 'source':row[header.index('Order Source')], 'total':row[header.index('Subtotal')], 'itemName':row[header.index('Item Name')], 'itemQty':row[header.index('Item Quantity')], 'itemTotal':row[header.index('Item Total')]} 
    with conn.cursor() as cur:
        date = row[header.index('Date')] + ' ' + row[header.index('Time')] 
        date = datetime.strptime(date, '%Y-%m-%d %I:%M %p')
        date = date.strftime('%Y-%m-%d %H:%M:%S')
        brandAndKitchen = colmap['brandANDkitchen']
        bAk = brandAndKitchen.split('-')
        brand = bAk[0]
        kitchen = bAk[1]
        if (orderalso):
            sql = "".join(["insert into AggOrder (orderId, billNumber, status, kitchen, source, brand, orderStartTime, orderEndTime, total, createdAt, updatedAt) values('", colmap['orderId'], "','" , colmap['billNumber'], "','", colmap['status'], "','",kitchen,"','",colmap['source'],"','",brand,"','", date,"','",date,"','",str(colmap['total']),"', current_timestamp(), current_timestamp())"])
            print (sql)
            try:
                cur.execute(sql)
            except:
                logger.error("ERROR: Failed to execute SQL ")
        sql = ''.join(["insert into AggOrderItems (orderId, billNumber, itemName, itemPrice, itemQuantity, createdAt, updatedAt) values('", colmap['orderId'],"','", colmap['billNumber'], "','", colmap['itemName'], "','" + str(colmap['itemTotal']) + "','", str(colmap['itemQty']),"', current_timestamp(), current_timestamp())"])
        print (sql)
        try:
            cur.execute(sql)
        except:
            logger.error("ERROR: Failed to execute SQL ")        
    conn.commit()

def procesSheet(book, connection):
    first_sheet = book.sheet_by_index(0)
    header = first_sheet.row_values(19)
    rows = first_sheet.nrows
    columns = len(header)
    print (header)
    print (rows)
    i = 20
    prevrow = header
    orderid = ""
    orderalso = True
    while i < rows:
        row = first_sheet.row_values(i)
        if (row[0] == ''):
            j = 0
            while (j < columns):
                if (row[j] == ''):
                    row[j] = prevrow[j]
                j = j + 1
        else:
            prevrow = row
        i = i + 1
        # print (row)
        if (orderid != row[header.index('Reference Number')]):
            orderalso = True
        else:
            orderalso = False
        orderid = row[header.index('Reference Number')]
        insertOrder(header, row, conn, orderalso)

In [10]:
list = [
  ["ZOMATO", "SS", "Brookfield", "0", "26-01-2019"], ["ZOMATO", "SS", "Hebbal", "3.5", "26-01-2019"], ["ZOMATO", "SS", "Kammanahalli", "0", "26-01-2019"], ["ZOMATO", "SS", "Malleshwaram", "0", "26-01-2019"], ["ZOMATO", "SS", "Whitefield", "3.5", "26-01-2019"], ["ZOMATO", "SS", "BTM", "0", "26-01-2019"], ["ZOMATO", "SS", "Kormangala", "0", "26-01-2019"], ["ZOMATO", "SS", "Panathur", "3.2", "26-01-2019"], ["ZOMATO", "SS", "Domlur", "0", "26-01-2019"], ["ZOMATO", "SS", "HSR", "0", "26-01-2019"], ["ZOMATO", "SS", "Sarjapur Road", "3.2", "26-01-2019"], ["ZOMATO", "IB", "Kaggdaspura", "3.5", "26-01-2019"], ["ZOMATO", "IB", "Hoodi", "0", "26-01-2019"], ["ZOMATO", "IB", "Brookfield", "3.3", "26-01-2019"]]
print (list)

[['ZOMATO', 'SS', 'Brookfield', '0', '26-01-2019'], ['ZOMATO', 'SS', 'Hebbal', '3.5', '26-01-2019'], ['ZOMATO', 'SS', 'Kammanahalli', '0', '26-01-2019'], ['ZOMATO', 'SS', 'Malleshwaram', '0', '26-01-2019'], ['ZOMATO', 'SS', 'Whitefield', '3.5', '26-01-2019'], ['ZOMATO', 'SS', 'BTM', '0', '26-01-2019'], ['ZOMATO', 'SS', 'Kormangala', '0', '26-01-2019'], ['ZOMATO', 'SS', 'Panathur', '3.2', '26-01-2019'], ['ZOMATO', 'SS', 'Domlur', '0', '26-01-2019'], ['ZOMATO', 'SS', 'HSR', '0', '26-01-2019'], ['ZOMATO', 'SS', 'Sarjapur Road', '3.2', '26-01-2019'], ['ZOMATO', 'IB', 'Kaggdaspura', '3.5', '26-01-2019'], ['ZOMATO', 'IB', 'Hoodi', '0', '26-01-2019'], ['ZOMATO', 'IB', 'Brookfield', '3.3', '26-01-2019']]


In [34]:
# Expected - [{'aggregator':'SWIGGY', 'brand':'WO', 'Brookfield':'4.1'
#                       ,'BTM':'4.1' ,'Domlur':'4.1' ,'Sarjapur Road':'4.1'
#                       ,'Panathur':'4.1' ,'Hebbal':'4.1' ,'Hoodi':'4.1'
#                       ,'HSR':'4.1' ,'Kammanahalli':'4.1' ,'Kaggdaspura':'4.1'
#                       ,'Kormangala':'4.1' ,'Malleshwaram':'4.1' ,'Whitefield':'4.1'}]
rtn = []
aggmap = {}
agg = ''
brand = ''
date = ''
for rec in list:
    print (rec)
    aggregator = rec[0]
    brand = rec[1]
    kitchen = rec[2]
    rating = rec[3]
    date = rec[4]
    #print (aggmap[str(aggregator)] == None)
    if (aggregator not in aggmap):
        aggmap[aggregator] = {}
    if (brand not in aggmap[aggregator]):
        aggmap[aggregator][brand] = {}
    aggmap[aggregator][brand][kitchen] = rating
    #aggmap['date'] = date
#print(aggmap)
for key,val in aggmap.items():
    for key1,val1 in val.items():
        entry = {}
        entry['aggregator'] = key
        entry['brand'] = key1
        entry['date'] = date
        for key2,val2 in val1.items():
            entry[key2] = val2
        rtn.append(entry)
print (rtn)
            

['ZOMATO', 'SS', 'Brookfield', '0', '26-01-2019']
['ZOMATO', 'SS', 'Hebbal', '3.5', '26-01-2019']
['ZOMATO', 'SS', 'Kammanahalli', '0', '26-01-2019']
['ZOMATO', 'SS', 'Malleshwaram', '0', '26-01-2019']
['ZOMATO', 'SS', 'Whitefield', '3.5', '26-01-2019']
['ZOMATO', 'SS', 'BTM', '0', '26-01-2019']
['ZOMATO', 'SS', 'Kormangala', '0', '26-01-2019']
['ZOMATO', 'SS', 'Panathur', '3.2', '26-01-2019']
['ZOMATO', 'SS', 'Domlur', '0', '26-01-2019']
['ZOMATO', 'SS', 'HSR', '0', '26-01-2019']
['ZOMATO', 'SS', 'Sarjapur Road', '3.2', '26-01-2019']
['ZOMATO', 'IB', 'Kaggdaspura', '3.5', '26-01-2019']
['ZOMATO', 'IB', 'Hoodi', '0', '26-01-2019']
['ZOMATO', 'IB', 'Brookfield', '3.3', '26-01-2019']
[{'aggregator': 'ZOMATO', 'brand': 'SS', 'date': '26-01-2019', 'Brookfield': '0', 'Hebbal': '3.5', 'Kammanahalli': '0', 'Malleshwaram': '0', 'Whitefield': '3.5', 'BTM': '0', 'Kormangala': '0', 'Panathur': '3.2', 'Domlur': '0', 'HSR': '0', 'Sarjapur Road': '3.2'}, {'aggregator': 'ZOMATO', 'brand': 'IB', 'date